# 第10章 表現学習

In [1]:
# これまでに定義した関数の読み込み

from chapter01 import get_string_from_file
from chapter02 import get_words_from_file, configure_fonts_for_japanese
from chapter03 import get_words, bows_to_cfs, load_aozora_corpus, get_bows,  add_to_corpus,\
    get_weights, translate_bows, get_tfidfmodel_and_weights
from chapter04 import vsm_search, get_list_from_file
from chapter05 import top_n, get_pr_curve, get_average_precision

## 10.3 Word2vec

In [2]:
# Listing 10.1 #

# 学習用の文書データ
training_documents = ['年越しには天ぷら蕎麦をいただきます',
                                   '関東では雑煮に鶏肉と小松菜を入れます']

# 名詞の並びに変換
training_data = [get_words(d) for d in training_documents]
print('training_data = ', training_data)

# ライブラリの読み込み
from gensim.models import Word2Vec

# 埋め込みを計算
w2v_model = Word2Vec(training_data, size=3, window=2, sg=1, min_count=1)

# ベクトルの表示
word = '蕎麦'
print(word, '=', w2v_model.wv[word])

training_data =  [['年越し', 'に', 'は', '天ぷら', '蕎麦', 'を', 'いただき', 'ます'], ['関東', 'で', 'は', '雑煮', 'に', '鶏肉', 'と', '小松菜', 'を', '入れ', 'ます']]
蕎麦 = [-0.13285957 -0.03071345 -0.03037666]


In [3]:
# Listing 10.2 #

from gensim.models import KeyedVectors

word_vectors =KeyedVectors.load('data/ch10/w2v.kv')

In [4]:
# Listing 10.3 #

from pprint import pprint

pprint(word_vectors.most_similar(positive=['酸素'],topn=5))

[('吸入', 0.8519949913024902),
 ('ヘモグロビン', 0.8410395383834839),
 ('水素', 0.837383508682251),
 ('熱量', 0.8194736242294312),
 ('試験管', 0.8179594278335571)]


In [5]:
# Listing 10.4 #

print(word_vectors.most_similar(positive=['饂飩', 'イタリア'], negative=['日本'], topn=1))

[('スパゲッティ', 0.5985097289085388)]


In [6]:
# Listing 10.5 #

data = [[['饂飩', 'イタリア'], ['日本']],
            [['信長', '美濃'], ['尾張']],
            [['丸の内', '大阪'], ['東京']]]

for p_words, n_words in data:
    top = word_vectors.most_similar(positive=p_words, negative=n_words, topn=1)
    
    # [('スパゲッティ', 0.5985)] から'スパゲッティ'を取り出す
    answer = top[0][0]
    
    print('{} - {} + {} = {}'.format(p_words[0], n_words[0], p_words[1], answer))

饂飩 - 日本 + イタリア = スパゲッティ
信長 - 尾張 + 美濃 = 道三
丸の内 - 東京 + 大阪 = 堂島


   ## 10.4 Doc2vec

In [7]:
# Listing 10.6 #

from gensim.models.doc2vec import TaggedDocument

tagged_data = [TaggedDocument(words=d, tags=[i]) for i, d in enumerate(training_data)]

pprint(tagged_data)

[TaggedDocument(words=['年越し', 'に', 'は', '天ぷら', '蕎麦', 'を', 'いただき', 'ます'], tags=[0]),
 TaggedDocument(words=['関東', 'で', 'は', '雑煮', 'に', '鶏肉', 'と', '小松菜', 'を', '入れ', 'ます'], tags=[1])]


In [8]:
# Listing 10.7 #

from gensim.models import Doc2Vec

# Doc2vecのモデルを作成
d2v_model = Doc2Vec(tagged_data, dm=1, vector_size=3, window=2, min_count=1)

In [9]:
# Listing 10.8 #

# タグ0を付与した文書のベクトルを表示
print(d2v_model.docvecs[0])

[ 0.14809698 -0.01563     0.02351491]


In [10]:
# Listing 10.9 #

# ダウンロードした学習済みモデルのファイル
# (ダウンロードしたファイルをディレクトリdataに置いた場合)
model_file = 'data/jawiki.doc2vec.dmpv300d.model'

# モデルの読み込み(時間がかかる)
d2v_wikipedia_model = Doc2Vec.load(model_file)

In [11]:
# Listing 10.10 #

# テキストを語の並びに変換
doc1 = get_words('Pythonを使うと自然言語処理が簡単にできる')
doc2 = get_words('実データを用いた情報検索プログラミングは楽しい')
print('doc1 = {}'.format(doc1))
print('doc2 = {}'.format(doc2))

# 学習済みモデルに基づくdoc1とdoc2の類似度の計算
print('類似度: %.4f ' % 
      d2v_model.docvecs.similarity_unseen_docs(d2v_wikipedia_model, doc1, doc2, steps=50))

doc1 = ['Python', 'を', '使う', 'と', '自然', '言語', '処理', 'が', '簡単', 'に', 'できる']
doc2 = ['実', 'データ', 'を', '用い', 'た', '情報', '検索', 'プログラミング', 'は', '楽しい']
類似度: 0.5479 


In [12]:
# Listing 10.11 #

def d2v_search(model, texts, query):
    # textsの各要素を名詞の並びに変換
    docs = [get_words(text) if type(text) is str else text for text in texts]

    # queryも変換
    query_doc = get_words(query) if type(query) is str else query

    # docs[i]とquery_docをベクトル化して類似度を計算
    # そして，(i, 類似度) をi番目の要素とするリストを作成
    r = [(i, model.docvecs.similarity_unseen_docs(model, doc,  query_doc, steps=50))
             for i, doc in enumerate(docs)]
    # 類似度に関して降順にソートしたもの（ランキング）を返す
    return sorted(r, key=lambda x: x[1], reverse=True)

In [13]:
# Listing 10.12 #

# 5章のデータの読み込み
book_texts = [get_string_from_file('data/ch05/%d.txt' % i) for i in range(10)]  

# クエリ
query = '人工知能'

# '人工知能'で検索した場合の正解（5章）
right_answer = [0, 1, 0, 1, 0, 1, 0, 0, 1, 1]

# 検索の実行
result = d2v_search(d2v_wikipedia_model, book_texts, query)

# 検索結果（ランキング）の表示
ranking = tuple([x[0] for x in result])
print(ranking)

# 平均適合率の計算と表示
print('%.4f' % get_average_precision(ranking, right_answer))

(3, 4, 8, 2, 0, 1, 9, 5, 6, 7)
0.6335


In [14]:
# Listing 10.13 #

import pickle
with open('data/ch10/tokenized.dat', 'rb') as f:
    tokenized_texts, tokenized_query = pickle.load(f)

result = d2v_search(d2v_wikipedia_model, tokenized_texts, tokenized_query)

ranking = tuple([x[0] for x in result])
print(ranking)
print('%.4f' % get_average_precision(ranking, right_answer))

(1, 8, 9, 5, 3, 0, 6, 2, 4, 7)
1.0000
